In [3]:
import bokeh.models
import bokeh.plotting
import bokeh.io
import bokeh.layouts
import bokeh.palettes
import bokeh.transform
import bokeh.embed

import sp_project.app_state
from sp_project.notebook_tools import default_clients
from sp_project.data_modelling.prediction_preparation import *


bokeh.io.output_notebook()

Loading BokehJS ...

### Get Data

In [4]:
lon = 7.70
lat = 46.42

In [5]:
async with default_clients() as (ow_client, db):
    app_state = sp_project.app_state.AppState(ow_client=ow_client, db_client=db)
    result = await extract_predictions_daily(app_state, lon=lon, lat=lat)

In [6]:
result

,temperature,wind_speed,rain,clouds
dt,,,,
2023-05-17 11:00:00+00:00,-7.45,2.98,0.00,100
2023-05-18 11:00:00+00:00,-3.21,2.27,0.00,100
2023-05-19 11:00:00+00:00,-1.21,2.71,0.00,99
2023-05-20 11:00:00+00:00,3.29,3.10,0.89,87
2023-05-21 11:00:00+00:00,3.98,2.60,2.94,98
2023-05-22 11:00:00+00:00,3.87,1.99,5.94,89
2023-05-23 11:00:00+00:00,2.13,2.24,2.81,90
2023-05-24 11:00:00+00:00,2.52,2.01,10.02,88


### Rain and average Temperature

In [7]:
day_source = bokeh.models.ColumnDataSource(data=result)

column_names = "avg_temp rain".split()

middle = result.index[len(result)//2]
selection_range = pd.Timedelta("2 days")

# Selected-Part-Plot
p = bokeh.plotting.figure(
    height=300, 
    width=800, 
    x_axis_type="datetime", 
    x_axis_location="above", 
    x_range=(middle-selection_range, middle+selection_range),
)

for tool in [bokeh.models.PanTool, bokeh.models.WheelZoomTool]:
    p.select_one(tool).dimensions = "width"


# Overview-Plot
overview = bokeh.plotting.figure(
    title="Drag the middle and edges of the selection box to change the range above",
    height=130, 
    width=800, 
    y_range=p.y_range,
    x_axis_type="datetime",
    y_axis_type=None,
    tools="hover", 
    toolbar_location=None, 
)

for fig in [p, overview]:
    fig.extra_y_ranges['rain'] = bokeh.models.Range1d(0, 5, bounds=(0, None))
    
    fig.vbar(
        x='dt',
        top="rain",
        source=day_source,    
        color="#0072B2",
        legend_label="Rain",
        width=pd.Timedelta(hours=20),
        y_range_name="rain",
    )

    fig.line(
        x='dt',
        y="temperature",
        source=day_source,    
        color="#D55E00",
        legend_label="Temperature",
    )

p.yaxis.axis_label = 'temperature [°C]'
p.yaxis.axis_label_text_color = "#D55E00"


rain_ax = bokeh.models.LinearAxis(y_range_name="rain", axis_label="rain [mm/h]")
rain_ax.axis_label_text_color ="#0072B2"
p.add_layout(rain_ax, 'right')
    
p.legend.location = "top_left"

range_tool = bokeh.models.RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

overview.ygrid.grid_line_color = None
overview.add_tools(range_tool)

line_fig = bokeh.layouts.column(p, overview)

bokeh.plotting.show(line_fig)

## Jupyter-Notebook Footer-Info

In [9]:
import os
import platform
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 22.4.0
Datetime: 2023-05-17 18:15:49
Python Version: 3.10.4
-----------------------------------
